# Chemical equation balancer

In [1]:
import sys
import re
import numpy as np

In [2]:
#  ==========================
#  = values =
#  ==========================

input_formula = "C1H1.4O0.247N0.000668"
input_Y_CO    = 0.03 # input_Y_CO
input_Y_s     = 0.09 # soot yield
input_X_H     = 0.1   # hydrogen atomic fraction in soot

input_O2      = 0.208
input_N2      = 0.783
input_H2O     = 0.00834
input_CO2     = 0.000387
input_CO      = 0
input_SOOT    = 0

formula = input_formula
y_s  = input_Y_s   # Y_soot
y_CO = input_Y_CO  # Y_CO
X_H  = input_X_H   # hydrogen atomic fraction in soot
bg_1 = 0.0000      # fuel in air
bg_2 = input_O2    # oxygen
bg_3 = input_N2    # nitrogen
bg_4 = input_H2O   # water H2O
bg_5 = input_CO2   # CO2
bg_6 = input_CO    # CO 
bg_7 = input_SOOT  # Soot

#  ==========================
#  = Parse chemical formula =
#  ==========================

C, H, O, N = 0, 0, 0, 0

# Search for C, H, O, N atoms in formula
match = re.search('[cC](\d+\.?\d*)', formula)
if match:
    C = match.group(1)
match = re.search('[hH](\d+\.?\d*)', formula)
if match:
    H = match.group(1)
match = re.search('[oO](\d+\.?\d*)', formula)
if match:
    O = match.group(1)
match = re.search('[nN](\d+\.?\d*)', formula)
if match:
    N = match.group(1)

# If an atom is included with no number following,
# then assign it a value of 1
if (C == 0) and ('C' in formula):
    C = 1
if (H == 0) and ('H' in formula):
    H = 1
if (O == 0) and ('O' in formula):
    O = 1
if (N == 0) and ('N' in formula):
    N = 1

# Convert all atom numbers to floats
C = float(C)
H = float(H)
O = float(O)
N = float(N)

# Throw error if formula does not include C or H
if ((C == 0) or (H == 0)):
    print("C and H must be included in the fuel")
    sys.exit()
#  ==========================================
#  = Chemical equation balance calculations =
#  ==========================================

### Based on work on a previous Matlab script by: ###
# Randall McDermott
# 3-31-11
# fds_simple_chemistry.m
#
# This script is a distilled version of the SIMPLE_CHEMISTRY routine in FDS and may be
# used as a check on reaction coefficients for primitive and lumped species.

# Molecular weights of C, H, O, N
MW_C = 12.0107
MW_H = 1.00794
MW_O = 15.9994
MW_N = 14.0067

# define the element matrix (number of atoms [rows] for each primitive species [columns])

i_fuel            = 0
i_oxygen          = 1
i_nitrogen        = 2
i_water_vapor     = 3
i_carbon_dioxide  = 4
i_carbon_monoxide = 5
i_soot            = 6

E = np.matrix([[C, 0, 0, 0, 1, 1, (1-X_H)],              # C
               [H, 0, 0, 2, 0, 0, X_H    ],              # H
               [O, 2, 0, 1, 2, 1, 0      ],              # O
               [N, 0, 2, 0, 0, 0, 0]     ], dtype=float) # N
     
MW = np.matrix([MW_C, MW_H, MW_O, MW_N]) # primitive species molecular weights

W = E.T * MW.T

# define the volume fractions of the background and fuel

v_0 = np.matrix([bg_1, bg_2, bg_3, bg_4, bg_5, bg_6, bg_7], dtype=float)
v_0 = v_0/np.sum(v_0) # normalize

v_1 = np.matrix([1, 0, 0, 0, 0, 0, 0], dtype=float)
v_1 = v_1/np.sum(v_1) # normalize

# the reaction coefficients for the product primitive species temporarily stored in v_2

v_2 = np.matrix([0, 0, 0, 0, 0, 0, 0], dtype=float)

# compute what we know so far

v_2[0,i_carbon_monoxide] = W.item(i_fuel) / W.item(i_carbon_monoxide) * y_CO
v_2[0,i_soot]            = W.item(i_fuel) / W.item(i_soot) * y_s

# linear system right hand side

b = E * (v_1.T - v_2.T)

# matrix

L = np.column_stack([E*v_0.T, E[:,i_carbon_dioxide], E[:,i_water_vapor], E[:,i_nitrogen]])

# solve the system

x = np.linalg.inv(L)*b

nu_0                    = x.item(0) # background stoichiometric coefficient
v_2.T[i_carbon_dioxide] = x.item(1)
v_2.T[i_water_vapor]    = x.item(2)
v_2.T[i_nitrogen]       = x.item(3)

nu_1 = -1 # fuel stoich coeff
nu_2 = np.sum(v_2) # prod stoich coeff

v_2 = v_2/nu_2 # normalize volume fractions

# display fuel properties

Z2Y = np.vstack([v_0, v_1, v_2])

Z2Y = Z2Y.T

coeff_fuel = Z2Y[0,1] # Fuel

coeff_lhs_1 = Z2Y[0,0] # Fuel
coeff_lhs_2 = Z2Y[1,0] # O2
coeff_lhs_3 = Z2Y[2,0] # N2
coeff_lhs_4 = Z2Y[3,0] # H2O
coeff_lhs_5 = Z2Y[4,0] # CO2
coeff_lhs_6 = Z2Y[5,0] # CO
coeff_lhs_7 = Z2Y[6,0] # C

coeff_rhs_1 = Z2Y[0,2] # Fuel
coeff_rhs_2 = Z2Y[1,2] # O2
coeff_rhs_3 = Z2Y[2,2] # N2
coeff_rhs_4 = Z2Y[3,2] # H2O
coeff_rhs_5 = Z2Y[4,2] # CO2
coeff_rhs_6 = Z2Y[5,2] # CO
coeff_rhs_7 = Z2Y[6,2] # C

if np.min(np.array([coeff_lhs_1, coeff_lhs_2, coeff_lhs_3, coeff_lhs_4, coeff_lhs_5, coeff_lhs_6, coeff_lhs_7,
              coeff_rhs_1, coeff_rhs_2, coeff_rhs_3, coeff_rhs_4, coeff_rhs_5, coeff_rhs_6, coeff_rhs_7])) < 0:
    print("Error: Results inculde negative stoichiometric coefficients")
    sys.exit()

Specify Heat of Combustion Based on Oxygen Consumption (Simple Chemistry)

$\Delta h \approx \frac{v_{O_2} W_{O_2}}{v_F W_F}$

In [21]:
coeff_rhs_2

np.float64(0.0)

In [17]:
wO2_ = 2 * MW_O
print(f'wO2: {wO2_}')

wO2: 31.9988


In [10]:
wF_ = W.item(i_fuel)
print(f'wF: {wF_}')

wF: 17.3830242756


### Balanced chemical equation

In [3]:
print("&SPEC ID         = 'FUEL_{0}',".format(formula))
print("      FORMULA    = '{0}'/".format(formula))

print("")

i = 1
if coeff_lhs_2 != 0:
    print("      SPEC_ID({1}) = 'OXYGEN',         VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_2,i))
    i = i+1
if coeff_lhs_3 != 0:
    print("      SPEC_ID({1}) = 'NITROGEN',       VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_3,i))
    i = i+1
if coeff_lhs_4 != 0:
    print("      SPEC_ID({1}) = 'WATER VAPOR',    VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_4,i))
    i = i+1
if coeff_lhs_5 != 0:
    print("      SPEC_ID({1}) = 'CARBON DIOXIDE', VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_5,i))
    i = i+1
if coeff_lhs_6 != 0:
    print("      SPEC_ID({1}) = 'CARBON MONOXIDE', VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_6,i))
    i = i+1
if coeff_lhs_7 != 0:
    print("      SPEC_ID({1}) = 'SOOT',           VOLUME_FRACTION({1})={0:.6f},".format(coeff_lhs_7,i))
    i = i+1
print("      BACKGROUND=.TRUE. /")

print("")

print("&SPEC ID         = 'PRODUCTS',")
i = 1
if coeff_rhs_2 != 0:
    print("      SPEC_ID({1}) = 'OXYGEN',          VOLUME_FRACTION(1})={0:.6f},".format(coeff_rhs_2,i))
    i = i+1
if coeff_rhs_3 != 0:
    print("      SPEC_ID({1}) = 'NITROGEN',        VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_3,i))
    i = i+1
if coeff_rhs_4 != 0:
    print("      SPEC_ID({1}) = 'WATER VAPOR',     VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_4,i))
    i = i+1
if coeff_rhs_5 != 0:
    print("      SPEC_ID({1}) = 'CARBON DIOXIDE',  VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_5,i))
    i = i+1
if coeff_rhs_6 != 0:
    print("      SPEC_ID({1}) = 'CARBON MONOXIDE', VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_6,i))
    i = i+1
if coeff_rhs_7 != 0:
    print("      SPEC_ID({1}) = 'SOOT',            VOLUME_FRACTION({1})={0:.6f},".format(coeff_rhs_7,i))
    i = i+1
print("/")
print("")
print("&REAC ID         = 'FUEL_{0}',".format(formula))
print("      FUEL       = 'FUEL_{0}',".format(formula))
print("      SPEC_ID_NU = 'FUEL_{0}','AIR','PRODUCTS',".format(formula))
print("      NU         =-{0:.6f},{1:.6f},{2:.6f},".format(coeff_fuel,nu_0,nu_2))
# ToDo
print("      HEAT_OF_COMBUSTION = 18700 /")

&SPEC ID         = 'FUEL_C1H1.4O0.247N0.000668',
      FORMULA    = 'C1H1.4O0.247N0.000668'/

      SPEC_ID(1) = 'OXYGEN',         VOLUME_FRACTION(1)=0.208057,
      SPEC_ID(2) = 'NITROGEN',       VOLUME_FRACTION(2)=0.783214,
      SPEC_ID(3) = 'WATER VAPOR',    VOLUME_FRACTION(3)=0.008342,
      SPEC_ID(4) = 'CARBON DIOXIDE', VOLUME_FRACTION(4)=0.000387,
      BACKGROUND=.TRUE. /

&SPEC ID         = 'PRODUCTS',
      SPEC_ID(1) = 'NITROGEN',        VOLUME_FRACTION(1)=0.699665,
      SPEC_ID(2) = 'WATER VAPOR',     VOLUME_FRACTION(2)=0.126174,
      SPEC_ID(3) = 'CARBON DIOXIDE',  VOLUME_FRACTION(3)=0.146399,
      SPEC_ID(4) = 'CARBON MONOXIDE', VOLUME_FRACTION(4)=0.003190,
      SPEC_ID(5) = 'SOOT',            VOLUME_FRACTION(5)=0.024571,
/

&REAC ID         = 'FUEL_C1H1.4O0.247N0.000668',
      FUEL       = 'FUEL_C1H1.4O0.247N0.000668',
      SPEC_ID_NU = 'FUEL_C1H1.4O0.247N0.000668','AIR','PRODUCTS',
      NU         =-1.000000,-5.212774,5.835724,
      HEAT_OF_COMBUSTION = 18700 /